In [137]:
! pip install -q transformers
! pip install -q sentencepiece
! pip install -q nltk
! git clone https://github.com/bhargav25dave1996/ICHCL_baseline.git
% cd /content/ICHCL_baseline

fatal: destination path 'ICHCL_baseline' already exists and is not an empty directory.
/content/ICHCL_baseline


In [2]:
! mkdir X_train
! mkdir X_val
! mkdir y_train
! mkdir y_val

In [3]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout


import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import stemmer as hindi_stemmer

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
english_stopwords = stopwords.words("english")
with open('final_stopwords.txt', encoding = 'utf-8') as f:
    hindi_stopwords = f.readlines()
    for i in range(len(hindi_stopwords)):
        hindi_stopwords[i] = re.sub('\n','',hindi_stopwords[i])
stopwords = english_stopwords + hindi_stopwords
english_stemmer = SnowballStemmer("english")

In [6]:
train_directories = []
for i in glob("data/train/*/"):
    for j in glob(i+'*/'):
        train_directories.append(j)

In [7]:
data = []
for i in train_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))
labels = []
for i in train_directories:
    with open(i+'labels.json', encoding='utf-8') as f:
        labels.append(json.load(f))

In [8]:
def tr_flatten(d,l):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
        'label':l[d['tweet_id']]
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] +' '+i['tweet'], #flattening comments(appending one after the other)
                'label':l[i['tweet_id']]
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] +' '+ i['tweet'] +' '+ j['tweet'], #flattening replies
                        'label':l[j['tweet_id']]
                    })
    return flat_text

def te_flatten(d):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] + i['tweet'],
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] + i['tweet'] + j['tweet'],
                    })
    return flat_text

In [9]:
data_label = []

for i in range(len(labels)):
    for j in tr_flatten(data[i], labels[i]):
        data_label.append(j)
train_len = len(data_label)

In [10]:
df = pd.DataFrame(data_label, columns = data_label[0].keys(), index = None)

In [11]:
df.head()

,tweet_id,text,label
0,1391715849979924484,@moonstar4u You have ugly mind and Botox and k...,HOF
1,1391730788421439496,@moonstar4u You have ugly mind and Botox and k...,NONE
2,1391770989566906368,@moonstar4u You have ugly mind and Botox and k...,HOF
3,1391718689020387332,@moonstar4u You have ugly mind and Botox and k...,HOF
4,1391950017485631490,@moonstar4u You have ugly mind and Botox and k...,HOF


In [12]:
df['label'].value_counts()

NONE    2899
HOF     2841
Name: label, dtype: int64

In [13]:
tweets = df.text
y = df.label

# Preprocessing

In [14]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_tweet(tweet):
    tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(regex_for_english_hindi_emojis,' ', tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)
    tokens = []
    for token in tweet.split():
        if token not in stopwords:
            token = english_stemmer.stem(token)
            token = hindi_stemmer.hi_stem(token)
            tokens.append(token)
    return " ".join(tokens)

In [15]:
cleaned_tweets = [clean_tweet(tweet) for tweet in tweets]

In [16]:
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(cleaned_tweets)
X = X.todense()

In [83]:
X = cleaned_tweets

In [18]:
y = y.to_list()

for i in range(len(y)):
    if y[i] == 'HOF':
        y[i] = 1
    else:
        y[i] = 0

# Training and evaluating model


In [84]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Masked LM (Loading Model)

In [20]:
from transformers import BertModel, BertTokenizer, BertForMaskedLM
import torch
from transformers import pipeline

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Sample Output

In [22]:
tokenized_input = tokenizer(
        X_train[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

print(sample_output)
print(sample_output.pooler_output.cpu().detach().numpy().shape)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1391,  0.0924, -0.2168,  ..., -0.5366,  0.3347,  0.3413],
         [ 0.5213,  0.4073, -0.1001,  ...,  0.1278,  0.9747, -0.2148],
         [ 0.3604, -0.5591,  0.4515,  ..., -0.1030,  0.3327, -0.3731],
         ...,
         [ 1.0797, -0.0793,  0.3736,  ..., -0.3190, -0.0960,  0.1562],
         [ 0.3741, -0.5151, -0.0887,  ..., -0.2985, -0.2406,  0.8990],
         [ 0.4231,  0.3185, -0.2946,  ..., -0.3219, -0.3105,  0.0410]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-0.8145, -0.5940, -0.9306,  0.8022,  0.7295, -0.2445,  0.7574,  0.4782,
         -0.7370, -1.0000, -0.1332,  0.8925,  0.9746,  0.6025,  0.8609, -0.6919,
         -0.2277, -0.6150,  0.4770, -0.0414,  0.7103,  1.0000,  0.1022,  0.3722,
          0.5993,  0.9753, -0.7114,  0.8509,  0.9328,  0.6764, -0.5748,  0.4518,
         -0.9818, -0.3724, -0.9393, -0.9832,  0.4785, -0.5332, -0.0813, -0.0368,
         -0.7609,  0.4650,  1.000

# Embedding the Training Data

In [23]:
print(len(X_train))

4592


In [24]:
input = []

for text in X_train:
    tokenized_input = tokenizer(
        text,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

print(len(input))

4592


# Saving X_train to disk

In [25]:
import pickle

In [26]:
output = []

j = 1

for i in range(len(input)):
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    if ((i + 1) % 200) == 0:
        file_name = "./X_train/output" + str(j) + ".txt"
        with open(file_name, "wb") as fp:   #Pickling
            pickle.dump(output, fp)
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./X_train/output" + str(j) + ".txt"
with open(file_name, "wb") as fp:   #Pickling
    pickle.dump(output, fp)
print(file_name + " done")
output = []

./X_train/output1.txt done
./X_train/output2.txt done
./X_train/output3.txt done
./X_train/output4.txt done
./X_train/output5.txt done
./X_train/output6.txt done
./X_train/output7.txt done
./X_train/output8.txt done
./X_train/output9.txt done
./X_train/output10.txt done
./X_train/output11.txt done
./X_train/output12.txt done
./X_train/output13.txt done
./X_train/output14.txt done
./X_train/output15.txt done
./X_train/output16.txt done
./X_train/output17.txt done
./X_train/output18.txt done
./X_train/output19.txt done
./X_train/output20.txt done
./X_train/output21.txt done
./X_train/output22.txt done
./X_train/output23.txt done


# Loading X_train from disk

In [138]:
output = []

for i in range(23):
    file_name = "./X_train/output" + str(i + 1) + ".txt"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

./X_train/output1.txt done
./X_train/output2.txt done
./X_train/output3.txt done
./X_train/output4.txt done
./X_train/output5.txt done
./X_train/output6.txt done
./X_train/output7.txt done
./X_train/output8.txt done
./X_train/output9.txt done
./X_train/output10.txt done
./X_train/output11.txt done
./X_train/output12.txt done
./X_train/output13.txt done
./X_train/output14.txt done
./X_train/output15.txt done
./X_train/output16.txt done
./X_train/output17.txt done
./X_train/output18.txt done
./X_train/output19.txt done
./X_train/output20.txt done
./X_train/output21.txt done
./X_train/output22.txt done
./X_train/output23.txt done


In [139]:
X_train = output
output = []

In [140]:
print(len(X_train))

4592


# Embedding the Testing Data

In [30]:
input = []

for text in X_val:
    tokenized_input = tokenizer(
        text,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

print(len(input))

1148


# Saving X_val to disk

In [31]:
output = []

j = 1

for i in range(len(input)):
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    if ((i + 1) % 200) == 0:
        file_name = "./X_val/output" + str(j) + ".txt"
        with open(file_name, "wb") as fp:   #Pickling
            pickle.dump(output, fp)
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./X_val/output" + str(j) + ".txt"
with open(file_name, "wb") as fp:   #Pickling
    pickle.dump(output, fp)
print(file_name + " done")
output = []

./X_val/output1.txt done
./X_val/output2.txt done
./X_val/output3.txt done
./X_val/output4.txt done
./X_val/output5.txt done
./X_val/output6.txt done


# Loading X_val from disk

In [141]:
output = []

for i in range(6):
    file_name = "./X_val/output" + str(i + 1) + ".txt"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

./X_val/output1.txt done
./X_val/output2.txt done
./X_val/output3.txt done
./X_val/output4.txt done
./X_val/output5.txt done
./X_val/output6.txt done


In [143]:
X_val = output
output = []
print(len(X_val))

1148


# Saving y_train & y_val to disk

In [34]:
with open("./y_train/y_train.txt", "wb") as fp:
    pickle.dump(y_train, fp)

with open("./y_val/y_val.txt", "wb") as fp:
    pickle.dump(y_val, fp)

# Loading y_train & y_val from disk

In [144]:
with open('./y_train/y_train.txt', "rb") as fp:
        y_train = pickle.load(fp)

with open('./y_val/y_val.txt', "rb") as fp:
        y_val = pickle.load(fp)

# Logistic Regression

In [168]:
X_train = np.array(X_train)
X_train = X_train.reshape(4592,768)

X_val = np.array(X_val)
X_val = X_val.reshape(1148, 768)

y_train = np.array(y_train)
y_val = np.array(y_val)

In [202]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [203]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.69      0.67       580
           1       0.67      0.63      0.65       568

    accuracy                           0.66      1148
   macro avg       0.66      0.66      0.66      1148
weighted avg       0.66      0.66      0.66      1148



# SVM

In [171]:
from sklearn.svm import SVC
classifier = SVC()
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [172]:
y_pred = classifier.predict(X_val)

In [173]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.60      0.62       580
           1       0.62      0.67      0.65       568

    accuracy                           0.64      1148
   macro avg       0.64      0.64      0.63      1148
weighted avg       0.64      0.64      0.63      1148



# Naive Bayes

In [174]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [175]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.95      0.67       580
           1       0.64      0.10      0.17       568

    accuracy                           0.53      1148
   macro avg       0.58      0.52      0.42      1148
weighted avg       0.58      0.53      0.42      1148



# Stochastic Gradient Descent

In [152]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier()
classifier.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [179]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.95      0.67       580
           1       0.64      0.10      0.17       568

    accuracy                           0.53      1148
   macro avg       0.58      0.52      0.42      1148
weighted avg       0.58      0.53      0.42      1148



# kNN

In [180]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [181]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.62      0.62       580
           1       0.62      0.62      0.62       568

    accuracy                           0.62      1148
   macro avg       0.62      0.62      0.62      1148
weighted avg       0.62      0.62      0.62      1148



# Decision Tree

In [188]:
from sklearn.tree import DecisionTreeClassifier

In [189]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [190]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.57      0.58       580
           1       0.57      0.58      0.57       568

    accuracy                           0.57      1148
   macro avg       0.57      0.57      0.57      1148
weighted avg       0.58      0.57      0.57      1148



# Random Forest

In [185]:
from sklearn.ensemble import RandomForestClassifier

In [186]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [187]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.65      0.66       580
           1       0.65      0.66      0.65       568

    accuracy                           0.66      1148
   macro avg       0.66      0.66      0.66      1148
weighted avg       0.66      0.66      0.66      1148



# Uploading Test data

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [54]:
% cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


# Testing data

In [87]:
test_directories = []
for i in glob("/content/drive/MyDrive/Colab Notebooks/subtask2_test/*/"):
    for j in glob(i+'*/'):
        test_directories.append(j)

In [88]:
test_directories

['/content/drive/MyDrive/Colab Notebooks/subtask2_test/religious controversies/1392808101007478794/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/religious controversies/1393528907102515200/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/wuhan virus/1392426204775391233/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/casteism/1395755293213003779/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/casteism/1394882097559400449/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/indian politics/1395657670762405888/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/indian politics/1396686927227588609/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/indian politics/1390362987995623428/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/israel/1392157687723347968/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/israel/1392413634773409792/',
 '/content/drive/MyDrive/Colab Notebooks/subtask2_test/covid crisis/138589100220

In [89]:
data = []
for i in test_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))

In [90]:
def te_flatten(d):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] + i['tweet'],
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] + i['tweet'] + j['tweet'],
                    })
    return flat_text

In [91]:
test_tweetid_data = []
#for test
for i in range(len(data)):
    for j in te_flatten(data[i]):
        test_tweetid_data.append(j)

In [92]:
test_df = pd.DataFrame(test_tweetid_data, columns = test_tweetid_data[0].keys(), index = None)

In [93]:
test_df.head()

,tweet_id,text
0,1392808101007478794,"In Gujarat, some believers have been going to ..."
1,1392808747928539137,"In Gujarat, some believers have been going to ..."
2,1392808849762000898,"In Gujarat, some believers have been going to ..."
3,1392836845361242121,"In Gujarat, some believers have been going to ..."
4,1392900440409337857,"In Gujarat, some believers have been going to ..."


In [94]:
test_tweets = test_df.text
tweet_ids = test_df.tweet_id

In [95]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_tweet(tweet):
    tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(regex_for_english_hindi_emojis,' ', tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)
    tokens = []
    for token in tweet.split():
        if token not in stopwords:
            token = english_stemmer.stem(token)
            token = hindi_stemmer.hi_stem(token)
            tokens.append(token)
    return " ".join(tokens)

In [96]:
X_test = [clean_tweet(tweet) for tweet in test_tweets]

# Loading Model

In [65]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [98]:
print(len(X_test))

1348


In [99]:
input = []

for text in X_test:
    tokenized_input = tokenizer(
        text,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

print(len(input))

1348


In [100]:
output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    if ((i + 1) % 200) == 0:
        file_name = "/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output" + str(j) + ".txt"
        with open(file_name, "wb") as fp:
            pickle.dump(output, fp)
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output" + str(j) + ".txt"
with open(file_name, "wb") as fp:
    pickle.dump(output, fp)
print(file_name + " done")
output = []

/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output1.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output2.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output3.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output4.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output5.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output6.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output7.txt done


In [115]:
output = []

for i in range(7):
    file_name = "/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output" + str(i + 1) + ".txt"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output1.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output2.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output3.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output4.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output5.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output6.txt done
/content/drive/MyDrive/Colab Notebooks/test_embed_feat/output7.txt done


In [116]:
X_test = output
output = []

In [118]:
X_test = np.array(X_test)
X_test.shape

(1348, 1, 768)

In [120]:
X_test = X_test.reshape(1348, 768)
X_test.shape

(1348, 768)

In [121]:
X_test = X_test.tolist()

In [122]:
import numpy as np

In [204]:
y_pred = classifier.predict(X_test)

In [205]:
y_pred

array([0, 0, 0, ..., 1, 1, 1])

In [206]:
test_df.head()

,tweet_id,text,label
0,1392808101007478794,"In Gujarat, some believers have been going to ...",HOF
1,1392808747928539137,"In Gujarat, some believers have been going to ...",HOF
2,1392808849762000898,"In Gujarat, some believers have been going to ...",NONE
3,1392836845361242121,"In Gujarat, some believers have been going to ...",HOF
4,1392900440409337857,"In Gujarat, some believers have been going to ...",HOF


In [207]:
y_test = y_pred

In [208]:
len(y_test)

1348

In [209]:
y_test = y_test.tolist()

In [210]:
for i in range(len(y_test)):
  if y_test[i] == 0:
    y_test[i] = 'NONE'
  else:
    y_test[i] = 'HOF'

In [211]:
y_test = np.array(y_test)

In [212]:
y_test

array(['NONE', 'NONE', 'NONE', ..., 'HOF', 'HOF', 'HOF'], dtype='<U4')

In [213]:
test_df['label'] = y_test

In [214]:
tweet_id = test_df['tweet_id']
tweet_id = np.array(tweet_id)
submission = {'tweet_id': tweet_id, 'label':y_test}
submission = pd.DataFrame(submission)
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/masked_lm_lr_submission.csv', index = False)